In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
# 1. 기존 리포지토리 폴더로 이동
import os
os.chdir('/content/drive/MyDrive/Codeit_AI_4th_Drug_image_CV_project')

In [3]:
# 2. 경로 확인
!pwd

/content/drive/MyDrive/Codeit_AI_4th_Drug_image_CV_project


In [4]:
import zipfile
import os
from pathlib import Path

In [7]:
def extract_zip_with_structure(zip_file_path, extract_to_path=None, verbose=True):
    """
    압축파일을 디렉토리 구조를 유지하며 압축해제하는 함수
    """

    # 압축파일 존재 여부 확인
    if not os.path.exists(zip_file_path):
        raise FileNotFoundError(f"압축파일을 찾을 수 없습니다: {zip_file_path}")

    # 압축해제 경로 설정
    if extract_to_path is None:
        extract_to_path = os.getcwd()

    # 압축해제 대상 폴더 생성
    Path(extract_to_path).mkdir(parents=True, exist_ok=True)

    try:
        with zipfile.ZipFile(zip_file_path, 'r') as zip_ref:
            # 압축파일 내용 확인
            file_list = zip_ref.namelist()

            if verbose:
                print(f"압축파일: {zip_file_path}")
                print(f"압축해제 경로: {extract_to_path}")
                print(f"총 파일/폴더 개수: {len(file_list)}개")
                print("=" * 50)

                # 최상위 폴더들 확인
                top_level_dirs = set()
                for item in file_list:
                    if '/' in item:
                        top_level_dirs.add(item.split('/')[0])
                    else:
                        if not item.endswith('.'):  # 파일인 경우
                            top_level_dirs.add(item)

                print(f"최상위 폴더들: {sorted(top_level_dirs)}")
                print("=" * 50)

            # 각 항목을 개별적으로 처리
            extracted_count = 0
            failed_items = []

            for item in file_list:
                try:
                    # 대상 경로 생성
                    target_path = os.path.join(extract_to_path, item)

                    # 디렉토리인 경우 (경로가 /로 끝나는 경우)
                    if item.endswith('/'):
                        Path(target_path).mkdir(parents=True, exist_ok=True)
                        if verbose and extracted_count < 10:  # 처음 10개만 출력
                            print(f"디렉토리 생성: {item}")
                    else:
                        # 파일인 경우
                        # 먼저 상위 디렉토리 생성
                        parent_dir = os.path.dirname(target_path)
                        if parent_dir:
                            Path(parent_dir).mkdir(parents=True, exist_ok=True)

                        # 파일 추출
                        with zip_ref.open(item) as source_file:
                            with open(target_path, 'wb') as target_file:
                                target_file.write(source_file.read())

                        if verbose and extracted_count < 10:  # 처음 10개만 출력
                            print(f"파일 추출: {item}")

                    extracted_count += 1

                except Exception as e:
                    failed_items.append((item, str(e)))
                    if verbose:
                        print(f"실패: {item} - {str(e)}")

            if verbose:
                print("=" * 50)
                print(f"압축해제 완료! 성공: {extracted_count}개, 실패: {len(failed_items)}개")

                if failed_items:
                    print("\n실패한 항목들:")
                    for item, error in failed_items:
                        print(f"  - {item}: {error}")

                # 압축해제된 폴더 구조 확인
                print("\n압축해제된 폴더 구조 확인:")
                check_extracted_structure(extract_to_path)

            return extract_to_path

    except zipfile.BadZipFile:
        raise ValueError(f"올바르지 않은 압축파일입니다: {zip_file_path}")
    except Exception as e:
        raise Exception(f"압축해제 중 오류가 발생했습니다: {str(e)}")

In [8]:
def check_extracted_structure(base_path, max_depth=3):
    """
    압축해제된 폴더 구조를 확인하는 함수
    """
    for root, dirs, files in os.walk(base_path):
        level = root.replace(base_path, '').count(os.sep)
        if level >= max_depth:
            continue

        indent = '  ' * level
        folder_name = os.path.basename(root) if os.path.basename(root) else 'Root'

        # 폴더 정보 출력
        file_count = len(files)
        dir_count = len(dirs)
        print(f"{indent}📁 {folder_name}/ (폴더: {dir_count}개, 파일: {file_count}개)")

        # 파일들 중 일부만 표시
        if files:
            subindent = '  ' * (level + 1)
            show_files = files[:3]  # 처음 3개만 표시
            for file in show_files:
                file_size = os.path.getsize(os.path.join(root, file))
                print(f"{subindent}📄 {file} ({file_size:,} bytes)")

            if len(files) > 3:
                print(f"{subindent}... 외 {len(files) - 3}개 파일")

In [9]:
def verify_specific_folders(base_path, expected_folders=['test_images', 'train_annotations', 'train_images']):
    """
    특정 폴더들이 제대로 생성되었는지 확인하는 함수
    """
    print("\n🔍 주요 폴더 존재 확인:")
    for folder in expected_folders:
        folder_path = os.path.join(base_path, folder)
        if os.path.exists(folder_path):
            if os.path.isdir(folder_path):
                file_count = sum(len(files) for _, _, files in os.walk(folder_path))
                print(f"✅ {folder}/ - 존재함 ({file_count}개 파일)")
            else:
                print(f"❌ {folder} - 파일로 존재함 (폴더가 아님)")
        else:
            print(f"❌ {folder}/ - 존재하지 않음")

In [10]:
if __name__ == "__main__":
    print("개선된 압축해제 함수가 정의되었습니다.")
    print("\n사용 예제:")
    print('# 기본 사용')
    print('extract_zip_with_structure("ai04-level1-project.zip")')
    print('\n# 특정 폴더에 압축해제 + 폴더 확인')
    print('extract_path = extract_zip_with_structure("ai04-level1-project.zip", "/content/dataset")')
    print('verify_specific_folders(extract_path)')
    extract_path = extract_zip_with_structure("./data/ai04_level1_project.zip", "./data")

개선된 압축해제 함수가 정의되었습니다.

사용 예제:
# 기본 사용
extract_zip_with_structure("ai04-level1-project.zip")

# 특정 폴더에 압축해제 + 폴더 확인
extract_path = extract_zip_with_structure("ai04-level1-project.zip", "/content/dataset")
verify_specific_folders(extract_path)
압축파일: ./data/ai04_level1_project.zip
압축해제 경로: ./data
총 파일/폴더 개수: 6858개
최상위 폴더들: ['test_images', 'train_annotations', 'train_images']
파일 추출: test_images/1.png
파일 추출: test_images/10.png
파일 추출: test_images/100.png
파일 추출: test_images/1003.png
파일 추출: test_images/1004.png
파일 추출: test_images/1005.png
파일 추출: test_images/1006.png
파일 추출: test_images/1007.png
파일 추출: test_images/1009.png
파일 추출: test_images/1010.png
압축해제 완료! 성공: 6858개, 실패: 0개

압축해제된 폴더 구조 확인:
📁 data/ (폴더: 4개, 파일: 2개)
  📄 data_explane.txt (0 bytes)
  📄 ai04_level1_project.zip (4,203,506,508 bytes)
  📁 .ipynb_checkpoints/ (폴더: 0개, 파일: 0개)
  📁 test_images/ (폴더: 0개, 파일: 843개)
    📄 1.png (1,891,360 bytes)
    📄 10.png (1,784,935 bytes)
    📄 100.png (1,690,579 bytes)
    ... 외 840개 파일
  📁 train_ann